In [103]:
%matplotlib inline
import matplotlib.pyplot as plt
from machinedesign.autoencoder.interface import load
import keras.backend as K
from keras.models import Model
import numpy as np
from skimage.io import imsave
from machinedesign.viz import grid_of_images_default

In [95]:
data = np.load('../results/jobs/04c1617e4181a6eb902d4299907d154e/gen/generated.npz')
X = data['generated']

In [96]:
model = load('../results/jobs/04c1617e4181a6eb902d4299907d154e')

In [97]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 28, 28)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 24, 24)       3328      
_________________________________________________________________
activation_1 (Activation)    (None, 128, 24, 24)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 20, 20)       409728    
_________________________________________________________________
activation_2 (Activation)    (None, 128, 20, 20)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 16, 16)       409728    
_________________________________________________________________
activation_3 (Activation)    (None, 128, 16, 16)       0         
__________

In [98]:
#mnist
inp = model.layers[0].input
hid_layer = model.get_layer('winner_take_all_spatial_1')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp], hid)
dec = K.function([hid], out)

In [99]:
im = grid_of_images_default(X[0:9])
imsave('out.png', im)

/home/ubuntu/miniconda2/envs/python3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [91]:
h = enc([X[0:9]])

In [100]:
im = grid_of_images_default(h[0])
imsave('out.png', im)

/home/ubuntu/miniconda2/envs/python3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [101]:
H = np.zeros((hid_layer.input_shape[1],) + hid_layer.input_shape[1:])
H[np.arange(H.shape[0]), np.arange(H.shape[0]), 2, 2] = 1000.0
x  = dec([H])
im = grid_of_images_default(x, normalize=True)
imsave('out.png', im)

/home/ubuntu/miniconda2/envs/python3/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [94]:
hid_layer.input_shape

(None, 128, 4, 4)

In [104]:
clf = load('../../discr/digits')
clf_enc = Model(inputs=clf.layers[0].input, outputs=clf.get_layer('p_re_lu_4').output)
#clf_enc = Model(inputs=clf.layers[0].input, outputs=clf.layers[0].output)

In [109]:
h = clf_enc.predict(X)